In [2]:
import pandas as pd
import numpy as np

In [3]:
from catboost import CatBoostRegressor

In [4]:
from catboost import CatBoostClassifier

In [5]:
from sklearn.metrics import mean_squared_error

In [6]:
loc_lbl = r"D:\TP2_Machine_Learning\Labels"

In [7]:
loc_ftr = r"D:\TP2_Machine_Learning\Features\ftr_ins"

In [8]:
loc_tr = r"D:\TP2_Machine_Learning\Training Sets"

In [9]:
ins_lb = pd.read_csv( loc_lbl + "\\ins_lb.csv", dtype = {"obs_18_20":"bool", "obs_21_23":"bool", "obs_24_26":"bool"} )

### Filtro de manera que tenga los datos necesarios

In [10]:
rh_18_20 = pd.read_csv( loc_tr + "\\rh_18_20_sc.csv" ); rh_18_20 = rh_18_20["ref_hash"].drop_duplicates().tolist()
rh_21_23 = pd.read_csv( loc_tr + "\\rh_21_23_sc.csv" ); rh_21_23 = rh_21_23["ref_hash"].drop_duplicates().tolist()

In [11]:
#rh_18_20 = pd.read_csv( r"D:\TP2_Machine_Learning\Training Sets\ins_18_20.csv" ).rename( columns = {"device_id":"ref_hash"} ); rh_18_20 = rh_18_20.drop_duplicates(subset=["ref_hash"])
#rh_21_23 = pd.read_csv( r"D:\TP2_Machine_Learning\Training Sets\ins_21_23.csv" ).rename( columns = {"device_id":"ref_hash"} ); rh_21_23 = rh_21_23.drop_duplicates(subset=["ref_hash"])

In [12]:
#rh_18_20 = rh_18_20.loc[:,['ref_hash']]
#rh_21_23 = rh_21_23.loc[:,['ref_hash']]
#rh_18_20.to_csv(r"D:\TP2_Machine_Learning\Training Sets\rh_18_20_sc.csv", index = False)
#rh_21_23.to_csv(r"D:\TP2_Machine_Learning\Training Sets\rh_21_23_sc.csv", index = False)

In [13]:
dt_18_20 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_18_20 ), ["ref_hash"] ] #Datos del (18-20)
sc_21_23 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_18_20 ), ["21_23_sc"] ] #SC del (21-23)

dt_21_23 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_21_23 ), ["ref_hash"] ] #Datos del (21-23)
sc_24_26 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_21_23 ), ["24_26_sc"] ] #SC del (24-26)

In [14]:
#sc_24_26

# <span style="color:orange">  Preparo los datos para predecir </span>
Con los datos 21-23 predecimos "24-26_sc"

In [479]:
set_trn = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_18_20 ), ["ref_hash","21_23_sc"] ]
set_tst = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_21_23 ), ["ref_hash", "24_26_sc"] ]

t_lim = 259200.00
max_t = 200

set_trn_loc = pd.concat( [ set_trn.loc[ set_trn["21_23_sc"] < t_lim ], set_trn.loc[ set_trn["21_23_sc"] == t_lim ].reset_index( drop = True ).loc[: max_t] ], axis = 0 ).sample(frac=1).reset_index(drop=True) 
set_tst_loc = pd.concat( [ set_tst.loc[ set_tst["24_26_sc"] < t_lim ], set_tst.loc[ set_tst["24_26_sc"] == t_lim ].reset_index( drop = True ).loc[: max_t] ], axis = 0 ).sample(frac=1).reset_index(drop=True) 

In [480]:
#x_clas = set_trn.loc[:,['ref_hash']].copy()
#y_clas = set_trn.loc[:,['21_23_sc']].copy()
#y_clas['compro'] = (y_clas['21_23_sc'] == y_clas['21_23_sc'].max()).astype('int8')

#del y_clas['21_23_sc']

#z_clas = set_tst.loc[:,['ref_hash']].copy()
#w_clas = set_tst.loc[:,['24_26_sc']].copy()
#w_clas['compro'] = (w_clas['24_26_sc'] == w_clas['24_26_sc'].max()).astype('int8')
#del w_clas['24_26_sc']

In [507]:
X_loc = set_trn_loc.loc[:,['ref_hash']] #Datos train
Y_loc = set_trn_loc.loc[:,['21_23_sc']] #Label train

Z_loc = set_tst_loc.loc[:,['ref_hash']] #Datos test
W_loc = set_tst_loc.loc[:,['24_26_sc']] #Label test

In [508]:
X_var = X_loc
Y_var = Y_loc
Z_var = Z_loc
W_var = W_loc

In [509]:
### Hacer features de ultima instalacion/subasta/evento, tiempo de instalacion (21-3, 4-10, 11-15, 16-20) y si hubieron subastas/eventos/clicks a la noche

In [510]:
ftr_01_Z = pd.read_csv( loc_ftr + "\\sin_hour_21_23.csv" ) 
ftr_02_Z = pd.read_csv( loc_ftr + "\\cant_ins_21_23.csv" ) #
ftr_03_Z = pd.read_csv( loc_ftr + "\\cins_imp_21_23.csv" )
#ftr_04_Z = pd.read_csv( loc_ftr + "\\cant_evt_21_23.csv" ) #<-bad
ftr_05_Z = pd.read_csv( loc_ftr + "\\cant_auc_21_23.csv" ) 
#ftr_06_Z = pd.read_csv( loc_ftr + "\\wifi_ins_21_23.csv" ) #<-bad
ftr_07_Z = pd.read_csv( loc_ftr + "\\main_app_21_23.csv" ) #
ftr_08_Z = pd.read_csv( loc_ftr + "\\type_ins_21_23.csv" ) 
ftr_09_Z = pd.read_csv( loc_ftr + "\\ref_type_21_23.csv" ) #
#ftr_10_Z = pd.read_csv( loc_ftr + "\\ip_encod_21_23.csv" ) #<-bad
ftr_11_Z = pd.read_csv( loc_ftr + "\\cevt_atr_21_23.csv" )
ftr_12_Z = pd.read_csv( loc_ftr + "\\kind_evt_21_23.csv" )
#ftr_13_Z = pd.read_csv( loc_ftr + "\\frst_auc_21_23.csv" ) #<-bad
ftr_14_Z = pd.read_csv( loc_ftr + "\\frst_evt_21_23.csv" ) #
ftr_15_Z = pd.read_csv( loc_ftr + "\\frst_ins_21_23.csv" ) #
ftr_16_Z = pd.read_csv( loc_ftr + "\\frst_clk_21_23.csv" ) #
ftr_17_Z = pd.read_csv( loc_ftr + "\\frst_auc_21_23.csv" ) #
#ftr_18_Z = pd.read_csv( loc_ftr + "\\hr_f_evt_21_23.csv" ) #<-bad

#ftr_19_Z = pd.read_csv( loc_ftr + "\\last_ins_21_23.csv" ) #<-bad
#ftr_20_Z = pd.read_csv( loc_ftr + "\\last_auc_21_23.csv" ) #<-bad
ftr_21_Z = pd.read_csv( loc_ftr + "\\last_evt_21_23.csv" )
#ftr_22_Z = pd.read_csv( loc_ftr + "\\last_clk_21_23.csv" ) #<-bad
#ftr_23_Z = pd.read_csv( loc_ftr + "\\me_mt_1_ins_21_23.csv" ) #<-bad
#ftr_24_Z = pd.read_csv( loc_ftr + "\\mt_1_ins_21_23.csv" ) #<-bad

#ftr_25_Z = pd.read_csv( loc_ftr + "\\me_ins_night_21_23.csv" ) #<-bad
#ftr_26_Z = pd.read_csv( loc_ftr + "\\ins_night_21_23.csv" ) #<-bad
#ftr_27_Z = pd.read_csv( loc_ftr + "\\me_ins_morn_21_23.csv" ) #<-bad
#ftr_28_Z = pd.read_csv( loc_ftr + "\\ins_morn_21_23.csv" ) #<-bad
#ftr_29_Z = pd.read_csv( loc_ftr + "\\me_ins_midday_21_23.csv" ) #<-bad
#ftr_30_Z = pd.read_csv( loc_ftr + "\\ins_midday_21_23.csv" ) #<-bad
#ftr_31_Z = pd.read_csv( loc_ftr + "\\me_ins_after_21_23.csv" ) #<-bad
#ftr_32_Z = pd.read_csv( loc_ftr + "\\ins_after_21_23.csv" ) #<-bad

#ftr_33_Z = pd.read_csv( loc_ftr + "\\me_auc_night_21_23.csv" ) #<-bad
#ftr_34_Z = pd.read_csv( loc_ftr + "\\auc_night_21_23.csv" ) #<-bad
#ftr_35_Z = pd.read_csv( loc_ftr + "\\me_auc_morn_21_23.csv" ) #<-bad
#ftr_36_Z = pd.read_csv( loc_ftr + "\\auc_morn_21_23.csv" ) #<-bad
#ftr_37_Z = pd.read_csv( loc_ftr + "\\me_auc_midday_21_23.csv" ) #<-bad
#ftr_38_Z = pd.read_csv( loc_ftr + "\\auc_midday_21_23.csv" ) #<-bad
#ftr_39_Z = pd.read_csv( loc_ftr + "\\me_auc_after_21_23.csv" ) #<-bad
#ftr_40_Z = pd.read_csv( loc_ftr + "\\auc_after_21_23.csv" ) #<-bad

#ftr_41_Z = pd.read_csv( loc_ftr + "\\me_evt_night_21_23.csv" ) #<-bad
#ftr_42_Z = pd.read_csv( loc_ftr + "\\evt_night_21_23.csv" ) #<-bad
#ftr_43_Z = pd.read_csv( loc_ftr + "\\me_evt_morn_21_23.csv" ) #<-bad
#ftr_44_Z = pd.read_csv( loc_ftr + "\\evt_morn_21_23.csv" ) #<-bad
#ftr_45_Z = pd.read_csv( loc_ftr + "\\me_evt_midday_21_23.csv" ) #<-bad
#ftr_46_Z = pd.read_csv( loc_ftr + "\\evt_midday_21_23.csv" ) #<-bad
#ftr_47_Z = pd.read_csv( loc_ftr + "\\me_evt_after_21_23.csv" ) #<-bad
#ftr_48_Z = pd.read_csv( loc_ftr + "\\evt_after_21_23.csv" ) #<-bad

#ftr_49_Z = pd.read_csv( loc_ftr + "\\me_clk_night_21_23.csv" ) #<-bad
#ftr_50_Z = pd.read_csv( loc_ftr + "\\clk_night_21_23.csv" ) #<-bad
#ftr_51_Z = pd.read_csv( loc_ftr + "\\me_clk_morn_21_23.csv" ) #<-bad
#ftr_52_Z = pd.read_csv( loc_ftr + "\\clk_morn_21_23.csv" ) #<-bad
#ftr_53_Z = pd.read_csv( loc_ftr + "\\me_clk_midday_21_23.csv" ) #<-bad
#ftr_54_Z = pd.read_csv( loc_ftr + "\\clk_midday_21_23.csv" ) #<-bad
#ftr_55_Z = pd.read_csv( loc_ftr + "\\me_clk_after_21_23.csv" ) #<-bad
#ftr_56_Z = pd.read_csv( loc_ftr + "\\clk_after_21_23.csv" ) #<-bad

#ftr_57_Z = pd.read_csv( loc_ftr + "\\me_mt_1_clk_21_23.csv" ) #<-bad
#ftr_58_Z = pd.read_csv( loc_ftr + "\\mt_1_clk_21_23.csv" ) #<-bad
#ftr_59_Z = pd.read_csv( loc_ftr + "\\me_mt_1_evt_21_23.csv" ) #<-bad
#ftr_60_Z = pd.read_csv( loc_ftr + "\\mt_1_evt_21_23.csv" ) #<-bad
#ftr_61_Z = pd.read_csv( loc_ftr + "\\me_mt_1_auc_21_23.csv" ) #<-bad
#ftr_62_Z = pd.read_csv( loc_ftr + "\\mt_1_auc_21_23.csv" ) #<-bad

ftr_63_Z = pd.read_csv( loc_ftr + "\\atbt_ins_21_23.csv" ) #


In [511]:
Z_var = Z_var.merge( ftr_01_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_02_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_03_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_04_Z, how = "inner", on = "ref_hash" ) #<- bad
Z_var = Z_var.merge( ftr_05_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_06_Z, how = "inner", on = "ref_hash" ) #<-bad
Z_var = Z_var.merge( ftr_07_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_08_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_09_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_10_Z, how = "inner", on = "ref_hash" ) #<-bad
Z_var = Z_var.merge( ftr_11_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_12_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_13_Z, how = "inner", on = "ref_hash" ) #<-bad
Z_var = Z_var.merge( ftr_14_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_15_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_16_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_17_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_18_Z, how = "inner", on = "ref_hash" ) #<-bad

#Z_var = Z_var.merge( ftr_19_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_20_Z, how = "inner", on = "ref_hash" ) #<-bad
Z_var = Z_var.merge( ftr_21_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_22_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_23_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_24_Z, how = "inner", on = "ref_hash" ) #<-bad

#Z_var = Z_var.merge( ftr_25_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_26_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_27_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_28_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_29_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_30_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_31_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_32_Z, how = "inner", on = "ref_hash" ) #<-bad

#Z_var = Z_var.merge( ftr_33_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_34_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_35_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_36_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_37_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_38_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_39_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_40_Z, how = "inner", on = "ref_hash" ) #<-bad

#Z_var = Z_var.merge( ftr_41_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_42_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_43_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_44_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_45_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_46_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_47_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_48_Z, how = "inner", on = "ref_hash" ) #<-bad

#Z_var = Z_var.merge( ftr_49_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_50_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_51_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_52_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_53_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_54_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_55_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_56_Z, how = "inner", on = "ref_hash" ) #<-bad

#Z_var = Z_var.merge( ftr_57_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_58_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_59_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_60_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_61_Z, how = "inner", on = "ref_hash" ) #<-bad
#Z_var = Z_var.merge( ftr_62_Z, how = "inner", on = "ref_hash" ) #<-bad

Z_var = Z_var.merge( ftr_63_Z, how = "inner", on = "ref_hash" )

# <span style="color:orange">  Preparo los datos para entrenar </span>

Con los datos 18-20 + "21-23_sc" entrenamos

In [512]:
#Y = sc_21_23

In [513]:
#X = dt_18_20

### Agregamos los features

In [514]:
ftr_01_X = pd.read_csv( loc_ftr + "\\sin_hour_18_20.csv" )
ftr_02_X = pd.read_csv( loc_ftr + "\\cant_ins_18_20.csv" )
ftr_03_X = pd.read_csv( loc_ftr + "\\cins_imp_18_20.csv" )
#ftr_04_X = pd.read_csv( loc_ftr + "\\cant_evt_18_20.csv" ) #<- bad
ftr_05_X = pd.read_csv( loc_ftr + "\\cant_auc_18_20.csv" )
#ftr_06_X = pd.read_csv( loc_ftr + "\\wifi_ins_18_20.csv" ) #<-bad
ftr_07_X = pd.read_csv( loc_ftr + "\\main_app_18_20.csv" )
ftr_08_X = pd.read_csv( loc_ftr + "\\type_ins_18_20.csv" )
ftr_09_X = pd.read_csv( loc_ftr + "\\ref_type_18_20.csv" )
#ftr_10_X = pd.read_csv( loc_ftr + "\\ip_encod_18_20.csv" ) #<-bad
ftr_11_X = pd.read_csv( loc_ftr + "\\cevt_atr_18_20.csv" )
ftr_12_X = pd.read_csv( loc_ftr + "\\kind_evt_18_20.csv" )
#ftr_13_X = pd.read_csv( loc_ftr + "\\frst_auc_18_20.csv" ) #<-bad
ftr_14_X = pd.read_csv( loc_ftr + "\\frst_evt_18_20.csv" )
ftr_15_X = pd.read_csv( loc_ftr + "\\frst_ins_18_20.csv" )
ftr_16_X = pd.read_csv( loc_ftr + "\\frst_clk_18_20.csv" )
ftr_17_X = pd.read_csv( loc_ftr + "\\frst_auc_18_20.csv" )
#ftr_18_X = pd.read_csv( loc_ftr + "\\hr_f_evt_18_20.csv" ) #<-bad

#ftr_19_X = pd.read_csv( loc_ftr + "\\last_ins_18_20.csv" ) #<-bad
#ftr_20_X = pd.read_csv( loc_ftr + "\\last_auc_18_20.csv" ) #<-bad
ftr_21_X = pd.read_csv( loc_ftr + "\\last_evt_18_20.csv" )
#ftr_22_X = pd.read_csv( loc_ftr + "\\last_clk_18_20.csv" ) #<-bad
#ftr_23_X = pd.read_csv( loc_ftr + "\\me_mt_1_ins_18_20.csv" ) #<-bad
#ftr_24_X = pd.read_csv( loc_ftr + "\\mt_1_ins_18_20.csv" ) #<-bad

#ftr_25_X = pd.read_csv( loc_ftr + "\\me_ins_night_18_20.csv" ) #<-bad
#ftr_26_X = pd.read_csv( loc_ftr + "\\ins_night_18_20.csv" ) #<-bad
#ftr_27_X = pd.read_csv( loc_ftr + "\\me_ins_morn_18_20.csv" ) #<-bad
#ftr_28_X = pd.read_csv( loc_ftr + "\\ins_morn_18_20.csv" ) #<-bad
#ftr_29_X = pd.read_csv( loc_ftr + "\\me_ins_midday_18_20.csv" ) #<-bad
#ftr_30_X = pd.read_csv( loc_ftr + "\\ins_midday_18_20.csv" ) #<-bad
#ftr_31_X = pd.read_csv( loc_ftr + "\\me_ins_after_18_20.csv" ) #<-bad
#ftr_32_X = pd.read_csv( loc_ftr + "\\ins_after_18_20.csv" ) #<-bad

#ftr_33_X = pd.read_csv( loc_ftr + "\\me_auc_night_18_20.csv" ) #<-bad
#ftr_34_X = pd.read_csv( loc_ftr + "\\auc_night_18_20.csv" ) #<-bad
#ftr_35_X = pd.read_csv( loc_ftr + "\\me_auc_morn_18_20.csv" ) #<-bad
#ftr_36_X = pd.read_csv( loc_ftr + "\\auc_morn_18_20.csv" ) #<-bad
#ftr_37_X = pd.read_csv( loc_ftr + "\\me_auc_midday_18_20.csv" ) #<-bad
#ftr_38_X = pd.read_csv( loc_ftr + "\\auc_midday_18_20.csv" ) #<-bad
#ftr_39_X = pd.read_csv( loc_ftr + "\\me_auc_after_18_20.csv" ) #<-bad
#ftr_40_X = pd.read_csv( loc_ftr + "\\auc_after_18_20.csv" ) #<-bad

#ftr_41_X = pd.read_csv( loc_ftr + "\\me_evt_night_18_20.csv" ) #<-bad
#ftr_42_X = pd.read_csv( loc_ftr + "\\evt_night_18_20.csv" ) #<-bad
#ftr_43_X = pd.read_csv( loc_ftr + "\\me_evt_morn_18_20.csv" ) #<-bad
#ftr_44_X = pd.read_csv( loc_ftr + "\\evt_morn_18_20.csv" ) #<-bad
#ftr_45_X = pd.read_csv( loc_ftr + "\\me_evt_midday_18_20.csv" ) #<-bad
#ftr_46_X = pd.read_csv( loc_ftr + "\\evt_midday_18_20.csv" ) #<-bad
#ftr_47_X = pd.read_csv( loc_ftr + "\\me_evt_after_18_20.csv" ) #<-bad
#ftr_48_X = pd.read_csv( loc_ftr + "\\evt_after_18_20.csv" ) #<-bad

#ftr_49_X = pd.read_csv( loc_ftr + "\\me_clk_night_18_20.csv" ) #<-bad
#ftr_50_X = pd.read_csv( loc_ftr + "\\clk_night_18_20.csv" ) #<-bad
#ftr_51_X = pd.read_csv( loc_ftr + "\\me_clk_morn_18_20.csv" ) #<-bad
#ftr_52_X = pd.read_csv( loc_ftr + "\\clk_morn_18_20.csv" ) #<-bad
#ftr_53_X = pd.read_csv( loc_ftr + "\\me_clk_midday_18_20.csv" ) #<-bad
#ftr_54_X = pd.read_csv( loc_ftr + "\\clk_midday_18_20.csv" ) #<-bad
#ftr_55_X = pd.read_csv( loc_ftr + "\\me_clk_after_18_20.csv" ) #<-bad
#ftr_56_X = pd.read_csv( loc_ftr + "\\clk_after_18_20.csv" ) #<-bad

#ftr_57_X = pd.read_csv( loc_ftr + "\\me_mt_1_clk_18_20.csv" ) #<-bad
#ftr_58_X = pd.read_csv( loc_ftr + "\\mt_1_clk_18_20.csv" ) #<-bad
#ftr_59_X = pd.read_csv( loc_ftr + "\\me_mt_1_evt_18_20.csv" ) #<-bad
#ftr_60_X = pd.read_csv( loc_ftr + "\\mt_1_evt_18_20.csv" ) #<-bad
#ftr_61_X = pd.read_csv( loc_ftr + "\\me_mt_1_auc_18_20.csv" ) #<-bad
#ftr_62_X = pd.read_csv( loc_ftr + "\\mt_1_auc_18_20.csv" ) #<-bad

ftr_63_X = pd.read_csv( loc_ftr + "\\atbt_ins_18_20.csv" )


In [515]:
X_var = X_var.merge( ftr_01_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_02_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_03_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_04_X, how = "inner", on = "ref_hash" ) #<- bad
X_var = X_var.merge( ftr_05_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_06_X, how = "inner", on = "ref_hash" ) #<-bad
X_var = X_var.merge( ftr_07_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_08_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_09_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_10_X, how = "inner", on = "ref_hash" ) #<-bad
X_var = X_var.merge( ftr_11_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_12_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_13_X, how = "inner", on = "ref_hash" ) #<-bad
X_var = X_var.merge( ftr_14_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_15_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_16_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_17_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_18_X, how = "inner", on = "ref_hash" ) #<-bad

#X_var = X_var.merge( ftr_19_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_20_X, how = "inner", on = "ref_hash" ) #<-bad
X_var = X_var.merge( ftr_21_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_22_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_23_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_24_X, how = "inner", on = "ref_hash" ) #<-bad

#X_var = X_var.merge( ftr_25_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_26_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_27_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_28_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_29_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_30_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_31_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_32_X, how = "inner", on = "ref_hash" ) #<-bad

#X_var = X_var.merge( ftr_33_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_34_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_35_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_36_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_37_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_38_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_39_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_40_X, how = "inner", on = "ref_hash" ) #<-bad

#X_var = X_var.merge( ftr_41_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_42_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_43_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_44_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_45_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_46_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_47_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_48_X, how = "inner", on = "ref_hash" ) #<-bad

#X_var = X_var.merge( ftr_49_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_50_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_51_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_52_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_53_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_54_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_55_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_56_X, how = "inner", on = "ref_hash" ) #<-bad

#X_var = X_var.merge( ftr_57_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_58_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_59_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_60_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_61_X, how = "inner", on = "ref_hash" ) #<-bad
#X_var = X_var.merge( ftr_62_X, how = "inner", on = "ref_hash" ) #<-bad

X_var = X_var.merge( ftr_63_X, how = "inner", on = "ref_hash" )

# <span style="color:orange">  Preparo las columnas </span>

In [518]:
X_var.head().T

,0,1,2,3,4
ref_hash,5.073551e+17,1.283142e+18,5.360746e+18,4.036648e+18,3.412192e+18
sin_hour,3.826834e-01,1.305262e-01,9.238795e-01,3.826834e-01,2.588190e-01
cant_ins,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
cant_imp,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
cant_auc,4.000000e+00,0.000000e+00,1.200000e+01,1.000000e+00,0.000000e+00
application_id,4.146730e-02,9.781023e-03,1.322781e-02,9.781023e-03,1.225047e-01
implicit_False,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
implicit_True,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
ref_type_mean,1.655094e-01,8.344906e-01,8.344906e-01,8.344906e-01,8.344906e-01
attributed,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [519]:
len(X_var.columns)

18

In [520]:
X_var['cant_ins'].value_counts()

1    3147
2     747
3     100
4      40
5       8
6       3
9       2
7       2
Name: cant_ins, dtype: int64

## Dropeamos los ref_hash (No estan codificados)

In [490]:
X_var = X_var.drop( "ref_hash", axis = 1 )
Z_var = Z_var.drop( "ref_hash", axis = 1 )

# Prediccion con <span style="color:green"> *CATBOOST*</span> 

In [47]:
#model = CatBoostClassifier(iterations=184,
#                          learning_rate=0.06,
#                          depth=2,
#                          l2_leaf_reg=0,
#                          random_strength=1,
#                          bagging_temperature=1)

#model.fit(X_var,Y_var)

### X: Train Data, Y: Train Labels, Z: Test Data, sc_24_26: Test Labels

### Record = iterations=300, learning_rate=0.06, depth=3, RMSE: 25506.739133

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
mse = make_scorer( mean_squared_error, greater_is_better = False ) 

In [ ]:
model = CatBoostRegressor(loss_function='RMSE')
parameters = {'depth'         : [2,3,6],
              'learning_rate' : [0.04, 0.06, 0.1],
              'iterations'    : [50, 150, 300]
                 }
grid = GridSearchCV(estimator=model, scoring=mse ,param_grid = parameters, cv = 12, n_jobs=2)
grid.fit(X_var, Y_var)  

In [86]:
    # Results from Grid Search
    print("\n========================================================")
    print(" Results from Grid Search " )
    print("========================================================")    
    
    print("\n The best estimator across ALL searched params:\n",
          grid.best_estimator_)
    
    print("\n The best score across ALL searched params:\n",
          grid.best_score_)
    
    print("\n The best parameters across ALL searched params:\n",
          grid.best_params_)
    
    print("\n ========================================================")


 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 -6547007371.867459

 The best parameters across ALL searched params:
 {'depth': 2, 'iterations': 150, 'learning_rate': 0.04}



In [ ]:
preds = grid.predict(Z_var)

### Parametros encontrados con grid-searchCV

## Record: iterations=184, learning_rate=0.06, depth=2, l2_leaf_reg=0, loss_function='RMSE' | RMSE: 80971.637759

In [498]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=184,
                          learning_rate=0.06,
                          depth=2,
                          l2_leaf_reg=0,
                          random_strength=1,
                          bagging_temperature=1,
                          boosting_type='Plain',
                          loss_function='RMSE')

In [499]:
# Fit model
model.fit(X_var, Y_var, verbose=False)

In [500]:
preds = model.predict(Z_var)

### Calculamos el RMSE

In [501]:
rmse = np.sqrt( mean_squared_error(W_var, preds) )
print("RMSE: %f" % (rmse) )

RMSE: 81046.140440


In [502]:
record = 81046.140440
#80971.637759
#76189.612816

In [503]:
W_var.head(10)

,24_26_sc
0,91801.564
1,187657.482
2,81827.588
3,259200.000
4,82591.862
5,190014.425
6,190231.268
7,71361.198
8,172974.531
9,1656.746


In [504]:
pd.DataFrame(preds).head(10)

,0
0,123898.208561
1,130397.479142
2,126252.845019
3,110043.815028
4,121921.362886
5,124193.878123
6,101169.305905
7,126933.594275
8,135445.302058
9,131266.787671


In [506]:
print("media " + str(pd.DataFrame(preds)[0].mean()))
print("std " + str(pd.DataFrame(preds)[0].std()))

media 124136.90546436826
std 9612.469383945052
